## Analise logs SATD

    
falta implementar    
    Analise do gprof 
    
        Por aproximação
               Por vídeo e configuração
                -- Funções mais utilizadas (mais chamadas) (por QP)
                   - com gráfico
                -- Funções que gastam mais tempo (por QP)


In [1]:
import scipy
from scipy import interpolate
import scipy.integrate as integrate
import math
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from IPython.display import clear_output,display, HTML
import numpy as np
import pandas as pd
import ipywidgets as widgets
import os
import glob


In [2]:
#CONSTANTS
GPROF_DIR = ".outputTests/gprof_log/"
VVCLOG_DIR =  "outputTests/vvc_log/"

In [3]:
#CLASSES
#class LogFunction: 
class LogVideo:
    def __init__(self, config, qp, name, path):
        self.config = config
        self.QP = qp
        self.name = name
        self.path = path
        #self.m_used_functions = []
        self.time = None
        self.bitrate = None
        self.BD_rate = None
        self.PSNR = None
            
    def get_config(self):
        return self.config
            
    def get_QP(self):
        return self.QP
        
    def get_name(self):   
        return self.name
    
    def set_time(self,time):
        self.time = time
       
    def get_time(self):
        return self.time
    
    #BITRATE
    def set_bitrate(self, bitrate):
        self.bitrate = bitrate
            
    def get_bitrate(self):
        return self.bitrate
            
    def set_PSNR(self, PSNR):
        self.PSNR = PSNR
            
    def get_PSNR(self):
        return self.PSNR
   
    #BDRATE 
    def set_bdrate(self, bdrate):
        self.BD_rate = bdrate
            
    def get_bdrate(self):
        return self.BD_rate
    
    
class Video:
    def __init__(self, name, path):
        self.name = name 
        self.path = path
        self.logs_AI = []
        self.logs_RA = []
        self.logs_LB = []
        self.BDRate_AI = None
        self.BDRate_RA = None
        self.BDRate_LB = None
        
        
    #manda os birates do self video, manda os PSNRs do self video
    #manda tambem o Video Precise pra comparar 
    #manda a configuracao pra setar o bdrate correto e pra comparar corretamente o video 
    def set_BDRate(self, config, VideoP):
        match config:
            case "AI":
                self.BDRate_AI = calc_bdrate(VideoP, self.get_bitrates(config), self.get_PSNRs(config), config); 
            case "RA":
                self.BDRate_RA = calc_bdrate(VideoP, self.get_bitrates(config), self.get_PSNRs(config), config); 
            case "LB":
                self.BDRate_LB = calc_bdrate(VideoP, self.get_bitrates(config), self.get_PSNRs(config), config); 
            case _:
                return false  
            
    def get_BDRate(self, config):
        match config:
            case "AI":
                return self.BDRate_AI
            case "RA":
                return self.BDRate_RA 
            case "LB":
                return self.BDRate_LB
            case _:
                return false          
        
    def get_name(self): 
        return self.name

    def get_path(self):
        return self.path
    
    def get_logs(self, config):
        match config:
            case "AI":
                return self.logs_AI
            case "RA":
                return self.logs_RA
            case "LB":
                return self.logs_LB
            case _:
                return false           
            
    def add_logs(self, config):
        path_logs = self.path + config + '/'
        
        #list logs in a specif configuration folder
        logs_list = os.listdir(path_logs)
        
        #fill config logs list
        #percorre os logs e vai preenchendo de acordo com o q ta na pasta 
        for log_item in logs_list:
            qp_item = log_item.split("_")[2]  
            qp_item_n = qp_item.split("p")[1]
            
            log_video = LogVideo(config,qp_item_n, log_item, path_logs)
        
            #aq eu preciso chamar outras funcoes pra preencher o resto dos dados dos logs
            #aquelas funcoes q pegam funcoes mais chamadas, tempo etc ect 
            get_data_vvc(path_logs + log_item, log_video)
    
            #coloca na lista
            match config:
                case "AI":
                    self.logs_AI.append(log_video)
                case "RA":
                    self.logs_RA.append(log_video)
                case "LB":
                    self.logs_LB.append(log_video) 
                    
    #GET BITRATES  
    #devolve np.array bitrates
    def get_bitrates(self, config):
        Rate1 = []
    
        match config:
            case "AI":
                for log in self.logs_AI:
                    Rate1.append(log.get_bitrate())
                
            case "RA":
                for log in self.logs_RA:
                    Rate1.append(log.get_bitrate())

            case "LB":
                for log in self.logs_LB:
                    Rate1.append(log.get_bitrate())
    
            case _:
                return false   
            
        return np.array(Rate1)
                
    #devolve np.array PSNRs 
    def get_PSNRs(self, config):
        PSNRs = []
        match config:
            case "AI":
                for log in self.logs_AI:
                    PSNRs.append(log.get_PSNR())
                
            case "RA":
                for log in self.logs_RA:
                    PSNRs.append(log.get_PSNR())

            case "LB":
                for log in self.logs_LB:
                    PSNRs.append(log.get_PSNR())
            case _:
                return false   
            
        return np.array(PSNRs)
    
    
    
class Aproximation:
    def __init__(self, size, aprox_type, path):
        self.size = size
        self.aprox_type = aprox_type
        self.path = path
        self.videos = []

    def add_video(self, video):
        self.videos.append(video)
        
    def get_size(self):
        return self.size
    
    def get_type(self):
        return self.aprox_type
    
    def get_path(self):
        return self.path
    
    #returns list
    def get_videos(self):
        return self.videos
    
    #returns a specific video
    def get_video(self, name):
        for video in self.videos:
            if (video.get_name() == name):
                return video
        return false 
                

class Matrix: 
    def __init__(self, size):
        self.size = size
        self.aprox_list = []
        
    #add a new aprox (-1, -2, -3...)
    def add_aprox(self, aprox):
        self.aprox_list.append(aprox)
        
    def get_aproximations(self):
        return self.aprox_list
    
    

In [4]:
#FUNCTIONS SYSTEM
#list all matrices
def list_matrices(terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix = item.split(terminator)[0]  
            if prefix not in names:  
                names.append(prefix)
    return names
    
#list all aproximations of a matrix researched
def list_aproximations(prefix, terminator, folder):
    items = os.listdir(folder)
    names = []
    for item in items:
        if os.path.isdir(os.path.join(folder, item)):
            prefix_item = item.split(terminator)[0]
            if prefix_item == prefix: 
                names.append(item)
    return names
    
    
#list all videos from a aprox folder
def list_videos(aprox_folder):
    videos = os.listdir(aprox_folder)
    return videos
    

In [5]:
#FUNCTIONS GPROF LOG
#returns the most used functions list 
def get_m_used_functions(file_gprof, number_of_itens):
    #--> ordenar funcoes de acordo com o nome (tipo aquele filtro) e devolver o objeto funcoes
    return void

 

In [6]:
#FUNCTIONS VVCLOG
def get_data_vvc(file_path, video_log):
    with open(file_path, 'r') as f:
        next = False
        for line in f:
            if "Total Frames" in line:
                next = True
            
            else: 
                if next:
                    data = ((line.strip().split("a")[1]).strip()).split()
                    
                    #teste = data.split()
                    video_log.set_bitrate(data[0])
                    
                    #PSNR YUV(??)
                    video_log.set_PSNR(data[4])

                    next = False
                    
            if "Total Time" in line:
                #elapsed(??)
                time = (line.strip().split()[2])
                video_log.set_time(time)

In [7]:
#FUNCTIONS PRINT TABLE

#make the table according the param
def print_table_vvc(matrix, videos, config, check_tags):
    
    aprox_list = []
    aproximations = matrix.get_aproximations()
    for aproximation in aproximations:
        if aproximation.get_type() != "Precise":
            aprox_list.append("-" + aproximation.get_type())
        else: 
            aprox_list.append(aproximation.get_type())
            
    
    videos_list = []
    tags_list = []
    qp_list = []
    
    for video in videos:
        for tag in check_tags:
            videos_list.append(video)
            videos_list.append(video)
            videos_list.append(video)
            videos_list.append(video)

        for tag in check_tags:
            qp_list.append(22)
        
        for tag in check_tags:
            tags_list.append(tag)
          
        for tag in check_tags:
            qp_list.append(27)
        
        for tag in check_tags:
            tags_list.append(tag)
        
        for tag in check_tags:
            qp_list.append(32)
        
        for tag in check_tags:
            tags_list.append(tag)
        
        for tag in check_tags:
            qp_list.append(37)
        
        for tag in check_tags:
            tags_list.append(tag)
    aprox_list.sort()

    #index table
    arrays = [
        np.array(videos_list),
        np.array(qp_list),
        np.array(tags_list),
    ]
    
    #content table
    data = []

    for aproximation in aproximations:
        #crio a lista pra aproximacao 
        data_column = []

        #pego os videos da respectiva aproximacao
        aprox_videos =  aproximation.get_videos()
        
        #pra cada video, pego os 4 logs da config escolhida
        logs_list = []
        for video in aprox_videos:
            
            logs_list = video.get_logs(config)
            
            #pra cada log, adiciono os 4 valores na lista 
            for log in logs_list:
                if 'Time' in check_tags:
                    data_column.append(log.get_time())
                
                if "Bitrate" in check_tags:
                    data_column.append(log.get_bitrate())
                
                if "PSNR" in check_tags:
                    data_column.append(log.get_PSNR())                    
        data.append(data_column)
    
    data = np.transpose(data)
    
    #display all table
    pd.set_option('display.max_rows', None)
    df = pd.DataFrame(np.array(data), index=arrays, columns=aprox_list)
    display(df)
    
def print_table_gprog():
    return void

In [8]:
#FUNCTIONS
#returns the analise
def analise(matrix_size, folder, config, check_tags):
    
    matrix = Matrix(matrix_size)
    aproximations_list = list_aproximations(matrix_size, "-",folder)

    for aproximation in aproximations_list:
        aprox_type = aproximation.split("-")[2]
        aprox_folder = folder + aproximation + "/"
        
        aprox = Aproximation(matrix_size, aprox_type, aprox_folder)
        
        videos_list = list_videos(aprox.get_path())
        for item in videos_list:
            video = Video(item, aprox.get_path() + item + '/')  
            
            configs_list = os.listdir(aprox.get_path() + item + '/')
            for item_config in configs_list:
                video.add_logs(item_config)
            
            aprox.add_video(video)
                    
        matrix.add_aprox(aprox)
        
    #add precise version
    path_precise = folder + "Precise/"
    precise = analise_precise(matrix_size, path_precise,config)
    matrix.add_aprox(precise)
    
    print_table_vvc(matrix, videos_list, config, check_tags)
    
    
    #bdrate
    aprox_list = matrix.get_aproximations()
    precise_video_list = precise.get_videos()
    
    
    #lista TODAS as aproximacoes dentro da matriz
    for aprox in aprox_list:
        videos = aprox.get_videos()
          
        #videos dentro dessa aproximacao
        for video in videos:    
            print("video aproximado: ")
            print(video.get_name())
            
            #compara com o PRECISE desse video 
            print("video preciso: ")
            VideoP = precise.get_video(video.get_name())
            print(VideoP.get_name())
            
            print("set bd rate: ")
            video.set_BDRate(config, VideoP)
            
            print("bd rate: ")
            print(video.get_BDRate(config))
            
    return matrix

    
#analise precise
def analise_precise(matrix_size, folder, config):
    #new object Aproximation
    precise = Aproximation(matrix_size, "Precise", folder)
        
    #fill aproximation videos list
    videos_list = list_videos(precise.get_path())
    for item in videos_list:
        video = Video(item, precise.get_path() + item + '/')  
            
        #fill logs for the video
        configs_list = os.listdir(precise.get_path() + item + '/')
        for item_config in configs_list:
            video.add_logs(item_config)
        precise.add_video(video)
    return precise
                    
    
    

In [15]:
#Funcoes METRICAS

# BD-Rate and BD-PNSR computation
# (c) Joao Ascenso (joao.ascenso@lx.it.pt)
def bj_delta(R1, PSNR1, R2, PSNR2, mode=0):
   
    print("debug R1 - bj")
    for rate in R1:
        print(rate)
        
    print("debug R2 - bj")
    for rate in R2:
        print(rate)
        
        
    lR1 = np.log(R1)
    lR2 = np.log(R2)

    # find integral
    if mode == 0:
        # least squares polynomial fit
        p1 = np.polyfit(lR1, PSNR1, 3)
        p2 = np.polyfit(lR2, PSNR2, 3)

        # integration interval
        min_int = max(min(lR1), min(lR2))
        max_int = min(max(lR1), max(lR2))

        # indefinite integral of both polynomial curves
        p_int1 = np.polyint(p1)
        p_int2 = np.polyint(p2)

        # evaluates both poly curves at the limits of the integration interval
        # to find the area
        int1 = np.polyval(p_int1, max_int) - np.polyval(p_int1, min_int)
        int2 = np.polyval(p_int2, max_int) - np.polyval(p_int2, min_int)

        # find avg diff between the areas to obtain the final measure
        avg_diff = (int2-int1)/(max_int-min_int)
    else:
        # rate method: sames as previous one but with inverse order
        p1 = np.polyfit(PSNR1, lR1, 3)
        p2 = np.polyfit(PSNR2, lR2, 3)

        # integration interval
        min_int = max(min(PSNR1), min(PSNR2))
        max_int = min(max(PSNR1), max(PSNR2))

        # indefinite interval of both polynomial curves
        p_int1 = np.polyint(p1)
        p_int2 = np.polyint(p2)

        # evaluates both poly curves at the limits of the integration interval
        # to find the area
        int1 = np.polyval(p_int1, max_int) - np.polyval(p_int1, min_int)
        int2 = np.polyval(p_int2, max_int) - np.polyval(p_int2, min_int)

        # find avg diff between the areas to obtain the final measure
        avg_exp_diff = (int2-int1)/(max_int-min_int)
        avg_diff = (np.exp(avg_exp_diff)-1)*100
    return avg_diff

#Recebe todos os np.arrays 
def calc_bdrate(VideoP, Rate2, PSNR2, config):
    #Rate 1 e 2 baseados no PRECISE 
    Rate1 = np.array(VideoP.get_bitrates(config))
    PSNR1 = np.array(VideoP.get_PSNRs(config))

    print("Video precise:")
    print(VideoP.get_name())
    
    bdrate = bj_delta(Rate1, PSNR1, Rate2, PSNR2, 1)
    return bdrate 

In [16]:
#FUNCTIONS BUTTON

#geral
def on_button_clear_click(b):
    clear_output()
    analise_vvc_print()
#vvc
def on_button_print_click(b):
    #listando todos os videos dentro das pastas das aproximacoes do valor aproximado
    ##AJUSTAR DEPOIS 
    check_tags = []
    if box1.value == True:
        check_tags.append(box1.description)
    if box2.value == True:
        check_tags.append(box2.description)
    if box3.value == True:
        check_tags.append(box3.description)

    analise(dropdown.value, VVCLOG_DIR, "AI",check_tags)  
    
 
    

 

In [17]:
# widgets VVC 
def analise_vvc_print():
    display(HTML("<h2>Analise VVC</h2>"))
    display(dropdown, container, buttons)

#widgets GPROF
def analise_gprof_print():
    display(HTML("<h1> Analise GPROF</h1>"))
    

In [18]:
# ELEMENTS 
# Dropdown
options_drop = list_matrices("-", VVCLOG_DIR)
options_drop.remove("Precise")
options_drop.sort()

dropdown = widgets.Dropdown(options=options_drop)

# Buttons
botao = widgets.Button(description='Imprimir')
botao.on_click(on_button_print_click)

botao_clear = widgets.Button(description='Clear')
botao_clear.on_click(on_button_clear_click)

buttons = widgets.HBox([botao, botao_clear])

# Selection
box1 = widgets.Checkbox(
    description='Time',
    disabled=False,
    layout=widgets.Layout(width='auto', margin='5px'),
    style={'description_width': 'initial'}
)
box2 = widgets.Checkbox(
    description='Bitrate',
    indent=False,
    disabled=False,
    layout=widgets.Layout(width='auto', margin ='5px'),
    style={'description_width': 'initial'}
)
box3 = widgets.Checkbox(
    description='PSNR',
    indent=False,
    disabled=False,
    layout=widgets.Layout(width='auto', margin='5px'),
    style={'description_width': 'initial'}
)

#caixa4 = widgets.Checkbox(description='BD-rate')


    
container = widgets.HBox([box1, box2, box3])

In [19]:
# MAIN
analise_vvc_print()


Dropdown(options=('16x8', '4x8', '8x16', '8x4'), value='16x8')

-1       -2  Precise
BasketballPass  22 PSNR  44.1883  44.1880  37.3694
                27 PSNR  34.5010  34.4900  40.7040
                32 PSNR  37.3734  40.6905  34.5133
                37 PSNR  40.7249  37.3669  44.1625
RaceHorses2     22 PSNR  32.6774  43.5616  39.6018
                27 PSNR  35.9356  39.6134  32.6773
                32 PSNR  39.6045  32.6746  35.9346
                37 PSNR  43.5668  35.9412  43.5658
BasketballDrive 22 PSNR  39.6339  38.1002  38.1065
                27 PSNR  41.0176  41.0162  39.6319
                32 PSNR  38.1006  39.6397  42.8503
                37 PSNR  42.8482  42.8475  41.0148
Campfire        22 PSNR  38.2939  44.4007  44.4025
                27 PSNR  40.8868  38.2941  38.2964
                32 PSNR  36.6555  40.8880  40.8885
                37 PSNR  44.4001  36.6585  36.6541
BQMall          22 PSNR  34.0440  43.0086  34.0517
                27 PSNR  37.0963  40.1320  43.0089
                32 PSNR  40.1251  37.0956  37.1102
                37 PSNR  43.0139  34.0502  40.1305
FourPeople      22 PSNR  37.7291  40.6176  43.2123
                27 PSNR  45.5510  37.7247  37.7305
                32 PSNR  40.6121  45.5500  40.6173
                37 PSNR  43.2095  43.2084  45.5445
RitualDance     22 PSNR  48.6996  46.2353  48.7002
                27 PSNR  43.5569  40.7665  43.5576
                32 PSNR  40.7613  48.6998  46.2423
                37 PSNR  46.2388  43.5575  40.7563
RaceHorses      22 PSNR  39.6045  43.5616  43.5658
                27 PSNR  43.5668  35.9412  35.9346
                32 PSNR  32.6774  39.6134  39.6018
                37 PSNR  35.9356  32.6746  32.6773
FoodMarket4     22 PSNR  44.1906  47.4940  47.4930
                27 PSNR  46.5622  46.5617  46.5624
                32 PSNR  47.4938  45.5659  44.1912
                37 PSNR  45.5667  44.1908  45.5658
Johnny          22 PSNR  41.7153  45.8270  39.4501
                27 PSNR  43.8245  41.7165  45.8272
                32 PSNR  45.8262  43.8215  41.7246
                37 PSNR  39.4388  39.4543  43.8232

video aproximado: 
BasketballPass
video preciso: 
BasketballPass
set bd rate: 
Video precise:
BasketballPass
debug R1 - bj
1608.4000
2904.5500
880.7000
4919.9000
debug R2 - bj
4938.4000
881.0000
1613.3500
2914.5000


TypeError: ufunc 'log' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
print("Test 1")

Rate1 = np.array([686.76, 309.58, 157.11, 85.95])
PSNR1 = np.array([40.28, 37.18, 34.24, 31.42])
Rate2 = np.array([893.34, 407.8, 204.93, 112.75])
PSNR2 = np.array([40.39, 37.21, 34.17, 31.24])

print("BD-PSNR: ", bj_delta(Rate1, PSNR1, Rate2, PSNR2, mode=0))
print("BD-RATE: ", bj_delta(Rate1, PSNR1, Rate2, PSNR2, mode=1))


Test 1
BD-PSNR:  -1.1848979217703248
BD-RATE:  31.39737405490941
